In [1]:
cd("/home/alir/Oceananigans.jl/")

In [2]:
using Pkg
pkg"activate ."

Activating environment at `~/Oceananigans.jl/Project.toml`


In [154]:
using Statistics
using OffsetArrays, FFTW
using Oceananigans, Oceananigans.Solvers

using Oceananigans: NoPenetrationBC, fill_halo_regions!
using Oceananigans.TimeSteppers: _compute_w_from_continuity!

In [155]:
@inline δx_caa(i, j, k, f) = @inbounds f[i+1, j, k] - f[i, j, k]
@inline δy_aca(i, j, k, f) = @inbounds f[i, j+1, k] - f[i, j, k]
@inline δz_aac(i, j, k, f) = @inbounds f[i, j, k+1] - f[i, j, k]

@inline ∂x_caa(i, j, k, Δx,  f) = δx_caa(i, j, k, f) / Δx
@inline ∂y_aca(i, j, k, Δy,  f) = δy_aca(i, j, k, f) / Δy
@inline ∂z_aac(i, j, k, ΔzF, f) = δz_aac(i, j, k, f) / ΔzF[k]

@inline ∂x²(i, j, k, Δx, f)       = (∂x_caa(i, j, k, Δx, f)  - ∂x_caa(i-1, j, k, Δx, f))  / Δx
@inline ∂y²(i, j, k, Δy, f)       = (∂y_aca(i, j, k, Δy, f)  - ∂y_aca(i, j-1, k, Δy, f))  / Δy
@inline ∂z²(i, j, k, ΔzF, ΔzC, f) = (∂z_aac(i, j, k, ΔzF, f) - ∂z_aac(i, j, k-1, ΔzF, f)) / ΔzC[k]

@inline div_ccc(i, j, k, Δx, Δy, ΔzF, u, v, w) = ∂x_caa(i, j, k, Δx, u) + ∂y_aca(i, j, k, Δy, v) + ∂z_aac(i, j, k, ΔzF, w)

@inline ∇²(i, j, k, Δx, Δy, ΔzF, ΔzC, f) = ∂x²(i, j, k, Δx, f) + ∂y²(i, j, k, Δy, f) + ∂z²(i, j, k, ΔzF, ΔzC, f)

∇² (generic function with 1 method)

In [156]:
function grid(zF)
    Nz = length(zF) - 1
    ΔzF = [zF[k+1] - zF[k] for k in 1:Nz]
    zC = [(zF[k] + zF[k+1]) / 2 for k in 1:Nz]
    ΔzC = [zC[k+1] - zC[k] for k in 1:Nz-1]
    return zF, zC, ΔzF, ΔzC
end

grid (generic function with 1 method)

In [157]:
arch = CPU()
Nx = Ny = 8
zF = [1, 2, 4, 7, 11, 16, 22, 29, 37]

9-element Array{Int64,1}:
  1
  2
  4
  7
 11
 16
 22
 29
 37

In [158]:
Lx, Ly, Lz = 100, 100, zF[end]
Δx, Δy = Lx/Nx, Ly/Ny

(12.5, 12.5)

In [159]:
Nz = length(zF) - 1
zF, zC, ΔzF, ΔzC = grid(zF)

# Need some halo regions.
ΔzF = OffsetArray([ΔzF[1], ΔzF...], 0:Nz)
ΔzC = [ΔzC..., ΔzC[end]]

# Useful for broadcasting z operations
ΔzC = reshape(ΔzC, (1, 1, Nz));

In [160]:
# Temporary hack: Useful for reusing fill_halo_regions! and BatchedTridiagonalSolver
# which only need Nx, Ny, Nz.
fake_grid = RegularCartesianGrid(size=(Nx, Ny, Nz), length=(Lx, Ly, Lz))

RegularCartesianGrid{Float64}
domain: x ∈ [0.0, 100.0], y ∈ [0.0, 100.0], z ∈ [-37.0, 0.0]
  resolution (Nx, Ny, Nz) = (8, 8, 8)
   halo size (Hx, Hy, Hz) = (1, 1, 1)
grid spacing (Δx, Δy, Δz) = (12.5, 12.5, 4.625)

In [161]:
#####
##### Generate batched tridiagonal system coefficients and solver
#####

function λi(Nx, Δx)
    is = reshape(1:Nx, Nx, 1, 1)
    @. (2sin((is-1)*π/Nx) / Δx)^2
end

function λj(Ny, Δy)
    js = reshape(1:Ny, 1, Ny, 1)
    @. (2sin((js-1)*π/Ny) / Δy)^2
end

kx² = λi(Nx, Δx);
ky² = λj(Ny, Δy);

In [162]:
# Lower and upper diagonals are the same
ld = [1/ΔzF[k] for k in 1:Nz-1]
ud = copy(ld)

# Diagonal (different for each i,j)
@inline δ(k, ΔzF, ΔzC, kx², ky²) = - (1/ΔzF[k-1] + 1/ΔzF[k]) - ΔzC[k] * (kx² + ky²)

d = zeros(Nx, Ny, Nz)
for i in 1:Nx, j in 1:Ny
    d[i, j, 1] = -1/ΔzF[1] - ΔzC[1] * (kx²[i] + ky²[j])
    d[i, j, 2:Nz-1] .= [δ(k, ΔzF, ΔzC, kx²[i], ky²[j]) for k in 2:Nz-1]
    d[i, j, Nz] = -1/ΔzF[Nz-1] - ΔzC[Nz] * (kx²[i] + ky²[j])
end

In [163]:
# Random right hand side
Ru = CellField(Float64, arch, fake_grid)
Rv = CellField(Float64, arch, fake_grid)
Rw = CellField(Float64, arch, fake_grid)

interior(Ru) .= rand(Nx, Ny, Nz)
interior(Rv) .= rand(Nx, Ny, Nz)
interior(Rw) .= zeros(Nx, Ny, Nz)
U = (u=Ru, v=Rv, w=Rw)

uv_bcs = HorizontallyPeriodicBCs()
w_bcs = HorizontallyPeriodicBCs(top=NoPenetrationBC(), bottom=NoPenetrationBC())

fill_halo_regions!(Ru.data, uv_bcs, arch, fake_grid)
fill_halo_regions!(Rv.data, uv_bcs, arch, fake_grid)

_compute_w_from_continuity!(U, fake_grid)

fill_halo_regions!(Rw.data,  w_bcs, arch, fake_grid)

R = zeros(Nx, Ny, Nz)
for i in 1:Nx, j in 1:Ny, k in 1:Nz
    R[i, j, k] = div_ccc(i, j, k, Δx, Δy, ΔzF, Ru.data, Rv.data, Rw.data)
end

@show sum(R)  # should be zero by construction.

F = zeros(Nx, Ny, Nz)
F = ΔzC .* R;  # RHS needs to be multiplied by ΔzC

sum(R) = 2.220446049250313e-16


In [164]:
#####
##### Solve system
#####

F̃ = fft(F, [1, 2])

btsolver = BatchedTridiagonalSolver(arch, dl=ld, d=d, du=ud, f=F̃, grid=fake_grid)

ϕ̃ = zeros(Complex{Float64}, Nx, Ny, Nz)
solve_batched_tridiagonal_system!(ϕ̃, arch, btsolver)

ϕ = CellField(Float64, arch, fake_grid)
interior(ϕ) .= real.(ifft(ϕ̃, [1, 2]))

@show sum(interior(ϕ));

sum(interior(ϕ)) = -1.2789769243681803e-13


In [165]:
#####
##### Compute Laplacian of solution ϕ to test that it's correct
#####

# Gotta fill halo regions
pbcs = HorizontallyPeriodicBCs()
fill_halo_regions!(ϕ.data, pbcs, arch, fake_grid)

∇²ϕ = CellField(Float64, arch, fake_grid)

for i in 1:Nx, j in 1:Ny, k in 1:Nz
    ∇²ϕ.data[i, j, k] = ∇²(i, j, k, Δx, Δy, ΔzF, ΔzC, ϕ.data)
end

In [166]:
interior(∇²ϕ) .- R

8×8×8 Array{Float64,3}:
[:, :, 1] =
 -5.55112e-17  -8.32667e-17  -3.33067e-16  …   4.16334e-17   0.0        
  2.498e-16     8.32667e-17   2.91434e-16      2.498e-16    -2.77556e-17
 -1.94289e-16   3.33067e-16   1.76942e-16     -5.89806e-17   2.22045e-16
  2.22045e-16  -5.55112e-17   0.0              8.32667e-17   2.77556e-17
 -1.9082e-17    2.22045e-16   0.0             -1.11022e-16  -3.46945e-17
 -7.28584e-17   2.77556e-17   0.0          …  -1.52656e-16  -1.31839e-16
 -2.08167e-16  -5.55112e-17   1.11022e-16     -4.44089e-16   5.68989e-16
 -2.498e-16    -2.35922e-16  -5.55112e-17     -8.32667e-17   2.22045e-16

[:, :, 2] =
  8.32667e-17   2.77556e-17   1.94289e-16  …  -1.52656e-16   7.32921e-17
 -1.83881e-16   0.0          -8.32667e-17     -9.71445e-17   9.71445e-17
  9.71445e-17  -4.16334e-16  -6.245e-17        4.85723e-17  -2.77556e-16
 -2.498e-16    -4.16334e-17  -1.80411e-16     -2.35922e-16  -1.38778e-17
 -6.59195e-17  -2.08167e-16  -2.77556e-17      1.07553e-16   8.67362e-17
  

In [169]:
interior(∇²ϕ) ≈ R

true